In [20]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import pandas_datareader as web
import datetime as dt
df = pd.read_csv ('saKaggle/ETH.csv')
articles = df.loc[:,"articles"]

In [4]:
from transformers import BertTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = TFAutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [5]:
def parse_string_list(input_string):
    # Remove the leading '[' and the trailing ']'
    trimmed = input_string[1:-1]

    # Initialize variables
    result = []
    temp = ''
    in_quotes = False
    quote_char = ''

    # Iterate through each character in the string
    for char in trimmed:
        if char in ("'", '"') and not in_quotes:  # Start of a new quoted string
            in_quotes = True
            quote_char = char
            temp += char  # Include the quote in the temp string
        elif char == quote_char and in_quotes:  # End of the current quoted string
            temp += char  # Include the quote in the temp string
            result.append(temp)  # Add the fully formed string to the result
            temp = ''  # Reset the temporary string holder
            in_quotes = False
        elif in_quotes:
            temp += char  # Add character to the temp string
        elif char == ',' and not in_quotes:  # Skip commas that are outside of quotes
            continue

    return result

# Sample input string
input_string = articles[5]

# Parse the string into a list
parsed_list = parse_string_list(input_string)



In [6]:
def find_sentiment(text):

    encoded_input = tokenizer(text, return_tensors='tf')
    outputs = model(encoded_input)

    predictions = tf.nn.softmax(outputs.logits, axis=-1)
    predicted_rating = tf.argmax(predictions, axis=1) + 1  # ratings from 1 to 5

    score = predicted_rating.numpy()[0]
    return score

In [7]:
print(find_sentiment(articles[0]))

1


0       ['Hacker Returns $26 Million Worth of Ethereum...
1       ['Algorithmia launches Ethereum-based AI compe...
2       ['Buying bitcoin and ethereum is like buying i...
3       ['Fusion Completes A Successful Ethereum Crowd...
4       ['TradeBlock adds ETH, LTC, XRP, BCH, and ZEC ...
                              ...                        
1821    ['Ethereum price rally hit crucial resistance ...
1822    ['Price analysis 2/22: BTC, ETH, BNB, XRP, ADA...
1823    ['What the Ethereum Shanghai Upgrade Will Mean...
1824    ['Price analysis 2/24: BTC, ETH, BNB, XRP, ADA...
1825    ['ETH Bulls Eye $1,650 on Justin Sun’s Bullish...
Name: articles, Length: 1826, dtype: object


In [21]:
sentiments = []
for x in articles:
    a = parse_string_list(x)
    b = []
    c = 0
    for n in a:
        b.append(find_sentiment(n))
    if len(b) > 0:
        c = sum(b)/len(b)
    sentiments.append(c)
print(sum(sentiments)/len(sentiments))
print(sentiments)

2.7976069172399924
[3.4, 3.7, 3.9, 3.1, 3.2, 1.6666666666666667, 3.2, 2.4, 2.6, 2.2, 3.4, 2.5, 2.6, 2.8, 3.6, 4.4, 2.6, 2.6, 2.8, 2.7, 2.1, 3.3, 2.7, 2.0, 3.4, 3.7, 1.4, 2.5, 2.5, 3.2, 2.5, 2.0, 3.0, 2.5, 1.9, 3.9, 2.7, 2.8, 2.9, 2.3, 3.2857142857142856, 3.0, 3.3, 3.3, 1.7, 2.9, 3.8, 2.4, 2.5, 4.4, 2.7, 3.5, 2.9, 3.0, 2.0, 2.8, 2.7, 2.2, 1.5, 3.4, 1.7, 3.3, 3.0, 3.8, 2.9, 3.0, 2.9, 1.9, 3.4, 3.3, 3.7, 3.4, 3.0, 3.2, 2.5, 2.7, 2.7, 3.1, 2.8, 3.0, 2.2, 3.2, 2.2, 3.9, 4.1, 2.5, 2.8, 2.6, 2.8, 2.1, 2.1, 2.3, 2.6, 4.1, 3.6, 3.7, 3.6666666666666665, 2.4, 2.4, 4.0, 3.8, 2.8, 3.3, 3.5, 2.8, 2.1, 2.3, 2.3, 2.2, 3.3, 3.0, 2.2, 4.0, 2.9, 2.6, 3.2, 2.7, 2.6, 2.4, 3.9, 3.3, 3.3, 3.2, 3.0, 2.5, 2.2, 3.2, 3.3, 3.0, 3.5, 3.4, 2.4, 1.5714285714285714, 4.4, 2.5, 3.0, 3.0, 3.9, 4.2, 2.625, 2.7, 3.0, 2.5, 2.7, 2.5, 3.4, 3.4, 3.3, 2.7, 2.2, 3.1, 2.9, 3.5, 1.3333333333333333, 3.4, 3.1, 3.6, 3.6, 3.1, 3.1, 2.8, 3.125, 3.1, 4.1, 3.5, 2.9, 3.1, 3.1, 2.7, 1.8, 2.5, 2.4, 2.9, 3.25, 3.4285714285714284, 3.6, 2.5, 

In [23]:
df['sentiments'] = sentiments
df
df.to_csv('kaggleSentiments.csv')